In [1]:

import pickle
from pythainlp.tokenize import word_tokenize
from gensim.models import KeyedVectors
import numpy as np

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pythainlp.corpus.common import thai_stopwords
import pandas as pd

from pythainlp import word_vector
from tqdm import tqdm


In [ ]:
model_original = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp
model = KeyedVectors.load_word2vec_format('/Users/thammasorn.h/Desktop/research-assist/model/LTW2V_v1.0-window5.bin', binary=True, unicode_errors='ignore')
data = pickle.load(open('entriesOct.pickle','rb'))
import re

def preprocess_word_list(word_list, mask="<NUM>"):
    word_list = [mask if re.search(r'\d', word) else word for word in word_list]
    word_list = [word for word in word_list if word not in thai_stopwords()]
    return word_list
data['preprocessed_Tokenized'] = data['Tokenized'].apply(preprocess_word_list)

In [21]:
pickle.dump(data['preprocessed_Tokenized'].tolist(), open('preprocessed_Tokenized.pickle','wb'))

In [ ]:
#%%
from gensim.models import Word2Vec,KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
import time
import os
import multiprocessing
import pickle 



class EpochLoggerAndSaver(CallbackAny2Vec):
    """Callback to log information and save the model at the end of each epoch."""

    def __init__(self, save_path, initial_epoch=0):
        self.epoch = initial_epoch
        self.save_path = save_path
        self.start_time = None

        # Create directory if it doesn't exist
        os.makedirs(self.save_path, exist_ok=True)

    def on_epoch_begin(self, model):
        self.start_time = time.time()
        print(f"Starting epoch {self.epoch + 1}")

    def on_epoch_end(self, model):
        elapsed_time = time.time() - self.start_time
        print(f"Finished epoch {self.epoch + 1} in {elapsed_time:.2f} seconds.")
        # Save the model
        # model_save_path = os.path.join(self.save_path, f"model_epoch_{self.epoch + 1}.model")
        model_save_path = os.path.join(self.save_path, f"model.model")
        model.save(model_save_path)
        model.wv.save_word2vec_format(model_save_path.replace('.model', '.bin'), binary=False)
        print(f"Model saved to {model_save_path}")

        self.epoch += 1

        
        
# %%
sentenceList_input = pickle.load(open('preprocessed_Tokenized.pickle','rb'))
cores=multiprocessing.cpu_count()
model = Word2Vec(min_count=6,window=9,vector_size=1000,workers=cores//2)
model.build_vocab(sentenceList_input)                
# model = Word2Vec.load('/home/thammasorn.h/ra/saved_model/model_epoch_69.model')
vocab_len=len(model.wv.index_to_key)
epoch_logger_and_saver = EpochLoggerAndSaver('saved_model',initial_epoch=0)
#%%
model.train(
    sentenceList_input,
    total_examples=model.corpus_count,
    epochs=300,
    callbacks=[epoch_logger_and_saver]
)
model.wv.save_word2vec_format('gensim_w2v_model.bin',binary=False)
# %%
